<a href="https://colab.research.google.com/github/oktaviacitra/deep-metric-learning/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

origin_path = "/content/drive"
drive.mount(origin_path)
folder_path = origin_path + "/MyDrive/Learning Journey/CIFAR10"

Mounted at /content/drive


In [2]:
from tensorflow.keras.datasets.cifar10 import load_data

(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
from tensorflow.image import per_image_standardization
from tensorflow.keras.utils import to_categorical

x_train, x_test = per_image_standardization(x_train), per_image_standardization(x_test)
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

In [ ]:
x = x_train.copy()
y = y_train.copy()

In [11]:
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits = 5)

VAL_ACC = []
VAL_LOSS = []
fold_count = 1

for train_index, val_index in kf.split(x,y):
  print(train_index.shape)

(40000,)
(40000,)
(40000,)
(40000,)
(40000,)


https://medium.com/the-owl/k-fold-cross-validation-in-keras-3ec4a3a00538